In [1]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [2]:
x = load_digits()['data']
y = load_digits()['target']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.33)

In [3]:
class DNN:
    
    def __init__(self, layers):
        self.w = self._init_w(layers)
        self.b = self._init_b(layers)
    
    # initialize weights
    def _init_w(self, layers):
        w = []
        for i in range(len(layers)-1):
            w.append(np.random.randn(layers[i], layers[i+1]))
        return w
    
    # initialize biases
    def _init_b(self, layers):
        b = []
        for i in range(len(layers)-1):
            b.append(np.random.randn(1, layers[i+1]))
        return b
    
    # activation
    def _activate(self, z):
        return 1.0 / (1.0 + np.exp(-z))
    
    # derivative of activation
    def _derivative(self, z):
        return self._activate(z) * (1-self._activate(z))
    
    # feed forward
    def _forward(self, x):
        x = x.astype('float32')
        z = [x.reshape(1, -1)]
        for i in range(len(self.w)):
            x = np.dot(x, self.w[i]) + self.b[i]
            z.append(x)
            x = self._activate(x)
        return x, z
    
    # backpropagation
    def _backprop(self, y, t, z):
        e = (y - t) * self._derivative(z[-1])
        delta_b = [e]
        delta_w = [self._activate(z[-2]).T @ e]
        for i in range(1, len(z)-1):
            e = np.dot(e, self.w[-i].T) * self._derivative(z[-(i+1)])
            delta_b.append(e)
            delta_w.append(self._activate(z[-(i+2)]).T @ e)
        return delta_w[::-1], delta_b[::-1]
    
    # evaluate the result
    def _evaluate(self, x, t):
        y, _ = self._forward(x)
        result = [(np.argmax(i), np.argmax(j)) for (i, j) in zip(y, t)]
        return sum(int(y == t) for (y, t) in result) / len(t)
    
    # train with batch
    def train(self, x, y, rate, epochs=100, batchsize=100):
        delta_w = [np.zeros(i.shape) for i in self.w]
        delta_b = [np.zeros(i.shape) for i in self.b]
        for i in range(epochs):
            idx = (np.random.sample(batchsize)*len(x)).astype(int)
            x_sample = x[idx]
            y_sample = y[idx]
            x_, z_ = self._forward(x_sample[0])
            delta_w, delta_b = self._backprop(x_, y_sample[0], z_)
            for j in range(1, len(y_sample)):
                x_, z_ = self._forward(x_sample[j])
                w_, b_ = self._backprop(x_, y_sample[j], z_)
                delta_w = [(ii + jj) for (ii, jj) in zip(delta_w, w_)]
                delta_b = [(ii + jj) for (ii, jj) in zip(delta_b, b_)]
            self.w = [(ii - rate*jj/len(x_sample)) for (ii, jj) in zip(self.w, delta_w)]
            self.b = [(ii - rate*jj/len(x_sample)) for (ii, jj) in zip(self.b, delta_b)]
            print (self._evaluate(x, y))

In [4]:
M = DNN([64, 16, 10])
ytt = OneHotEncoder().fit_transform(y_train.reshape(-1,1)).toarray()

In [5]:
M.train(x_train, ytt, 3, 1000, 10)

0.09808811305070657
0.10224438902743142
0.10058187863674148
0.09975062344139651
0.11637572734829593
0.12718204488778054
0.13632585203657524
0.12635078969243557
0.11221945137157108
0.0889443059019119
0.09891936824605153
0.10058187863674148
0.11055694098088113
0.1172069825436409
0.12053200332502079
0.11970074812967581
0.12136325852036575
0.11886949293433084
0.12967581047381546
0.13881961762261014
0.13798836242726517
0.1230257689110557
0.13549459684123025
0.13300083125519535
0.1371571072319202
0.1404821280133001
0.1313383208645054
0.13798836242726517
0.1487946799667498
0.15544472152950956
0.16043225270157938
0.17206982543640897
0.17206982543640897
0.15876974231088944
0.15544472152950956
0.17539484621778886
0.18204488778054864
0.18952618453865336
0.18786367414796343
0.19534497090606817
0.19451371571072318
0.20116375727348296
0.19035743973399832
0.20199501246882792
0.200332502078138
0.19866999168744803
0.2111388196176226
0.20615128844555278
0.19035743973399832
0.19118869492934332
0.18620116